In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Simulating random walks ##

When we discussed the Press-Schechter approach to the halo mass function we found that we needed to compute the statistics of a random walk with an absorbing barrier.  This was relatively easy to do by treating the problem as a diffusion equation solved with the method of images.  This is also quite an easy problem to simulate, and that allows us to handle more complex situations (such as curved barriers) and other problems.

Random walks of this sort show up in lots of places, and one of the interesting things about them is the generation of power-law tails in the distribution function.  Other places this sort of behavior comes up is in the distribution of the sizes of glacial lakes, distributions of atom "traps" in materials or various dynamics problems.

A classic "example" of a random walk near a barrier is drunks leaving a pub near the ocean, or a canal or river.

## Drunk near a canal ##

Consider an ensemble of drunks leaving a pub ($x=0$), each moving in a single dimension.  We assume that at every step the drunk moves left or right by 1 unit, with the direction being chosen at random.  If there are no constraints, then after $N$ steps the distribution of final locations will be Gaussian (in the limit of large $N$) with a dispersion of $\sqrt{N}$.  Note that due to the central limit theorem, if the left-right steps are really random then in the large $N$ limit it won't matter if we choose unit steps or steps chosen from a Gaussian or any other distribution with finite mean and variance (recall the derivation of the central limit theorem using the characteristic function).

In [ ]:
# Generate a series of left-right choices, then sum them to
# get the final position.
def final_position(Nwalker=1000,Nstep=100):
    leftright = np.random.choice([-1.0,1.0],size=(Nwalker,Nstep))
    finalpos  = np.sum(leftright,axis=1)
    return(finalpos)

In [ ]:
# Make a histogram of the results, and compare this
# to a Gaussian of \sqrt{N}.
#
fig,ax = plt.subplots(1,1,figsize=(6,4))
nbin   = 25
#
Nw = 1000
Ns = 500
fp = final_position(Nw,Ns)
ax.hist(fp,bins=nbin,density=True,alpha=0.5)
#
xx    = np.linspace(-150,150,1000)
sig   = np.sqrt(float(Ns))
gauss = 1./np.sqrt(2*np.pi)/sig*np.exp(-0.5*(xx/sig)**2)
ax.plot(xx,gauss)
#
Nw = 1000
Ns = 2000
fp = final_position(Nw,Ns)
ax.hist(fp,bins=nbin,density=True,alpha=0.5)
#
xx    = np.linspace(-150,150,1000)
sig   = np.sqrt(float(Ns))
gauss = 1./np.sqrt(2*np.pi)/sig*np.exp(-0.5*(xx/sig)**2)
ax.plot(xx,gauss)

### With a constraint ###

Ok, so now let's put in a constraint -- in our case this will be a "canal" at $x=30$.  If the drunk reaches the canal they are removed from the problem.  With our Monte-Carlo approach this is almost completely trivial ...

In [ ]:
# Add a canal at x_canal.
#
x_canal = 30.
#
fig,ax = plt.subplots(1,1,figsize=(6,4))
nbin   = 25
#
Nw = 1000
Ns = 500
fp = final_position(Nw,Ns)
fp = fp[fp<x_canal]
ax.hist(fp,bins=nbin,density=True)
# Note we don't need to fiddle with the normalization
# or anything because "hist" is doing it for us.
#
# We'll plot the Gaussian for comparison, although clearly
# we don't expect agreement in this case!
xx    = np.linspace(-150,150,1000)
sig   = np.sqrt(float(Ns))
gauss = 1./np.sqrt(2*np.pi)/sig*np.exp(-0.5*(xx/sig)**2)
ax.plot(xx,gauss)
#

In [ ]:
#
# Compare the distribution above to the analytic expectation derived
# as in class for the Press-Schechter problem.
#

## Heavy tails ##

Now let us show that the distribution of returns times (the number of steps required to return to the same level) has a power-law tail, specifically that the probability that the random walk returns to $0$ after $n$ steps has a power-law.  Power-law distributions are of great interest in complexity theory, network theory, self-organized criticality etc.  For example, we can use the above to argue that the random walk model implies the distribution of glacial lakes should be a power-law (as it is observed to be).  Ignoring the problem of "lakes within lakes".

This random walk model is actually very interesting from a number of different aspects.  The insights we need all arise from Pascal's triangle, or the binomial distribution.  For $n$ steps the number of walks is $2^n$. The number of ways of getting to zero is the number of ways of choosing equal numbers of "up" and "down" steps, given equal probability, i.e.~$n$ choose $n/2$ ($n$ must be even). The probability that the $n^{\rm th}$ step arives at zero is thus
\begin{equation}
  P(S_n=0) = 2^{-n} \left(\begin{array}{c} n \\ n/2 \end{array}\right)
           = 2^{-n} \frac{n!}{(n/2)!(n/2)!}
  \to \left(\frac{2}{n\pi}\right)\quad {\rm as}\ n\to\infty
\end{equation}
where we have made use of the independence of each step and used Stirling's formula for the asymptotic limit. (In probability theory the idea that past events can't help predict the future is known as a "martingale", in the use of Stirling's formula you need to include the $\mathcal{O}(\ln n)$ term.) Glacial lakes are formed by the motion of glaciers over a landscape. As it moves the glacier can dip down and dig more deeply into the surface, or it can rise up and dig less deeply.  A lake is formed in a depression bounded by two regions of the same height.  This size of the lake is clearly given by the return time distribution of a random walk -- as observed.  There is a well-known problem in the Press-Schechter theory known as the "halo within halo" problem (analogous to the "lakes within lakes" issue!).  When doing the accounting I need to not count a low mass halo within a region which will collapse to form a larger mass halo.  The random walk picture does this for us, but if I try to go directly from the statistics of the initial field this is more difficult.

In [ ]:
#
# Modify the code above to compute the P(0) as a function of Nstep and
# show it has a power-law behavior.  Note if you choose your array of
# options to be [-1,1] rather than [-1.0,1.0] as I did then you can
# avoid any floating point roundoff error.
#